In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
#%matplotlib inline

In [2]:
import math

def grayscale(img):
    """
    Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')
    """
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel""",
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with",
    mask = np.zeros_like(img)
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by \"vertices\" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.   
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
                

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
    
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=    max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img, lines
    

def calculate_lane_param(lines, slope_sign=None):
    """
    Divide the left or right lanes based on the sign of the slope.
    The 'True' slope/intercept = the MEDIAN value. This is based on
    intuition from the pepper&salt noise.
    
    param:  slope_sign = 'negative' or 'positive'
    """
    slopes = []
    intercepts = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2 - y1) / ( x2 - x1 )
            intercept = y1 - slope * x1
            if slope == np.inf or slope == -np.inf or intercept == np.inf or intercept == -np.inf:
                pass
            elif slope_sign == 'negative' and slope < 0:
                slopes.append(slope)
                intercepts.append(intercept)
            elif slope_sign == 'positive' and slope > 0:
                slopes.append(slope)
                intercepts.append(intercept)
    # Use 'median' instead of 'average' to remove outliers.
    slope = np.median(slopes)
    intercept = np.median(intercepts)
    #print('Differece: ', slope-np.mean(slopes))

    return slope, intercept


def lane_lines(img, lines, x_left, x_right):
    """
    `img` any image to extract the shape
    `lines` are the hough lines.
    `x_left` and `x_right` defines the top points of the lane lines.
    """
    # Calculate the lane param
    left_slope, left_intercept = calculate_lane_param(lines, slope_sign='negative')
    right_slope, right_intercept = calculate_lane_param(lines, slope_sign='positive')
    
    # Draw the lanes
    lane_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    cv2.line(lane_img, (x_left, int(left_slope * x_left + left_intercept)),
                       (0, int(left_intercept)),
                       (255, 0, 0), 10)
    cv2.line(lane_img, (x_right, int(right_slope * x_right + right_intercept)), 
                       (img.shape[1], int(right_slope * img.shape[1] + right_intercept)),
                       (255, 0, 0), 10)
    return lane_img

                
# Python 3 has support for cool math symbols.
def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [3]:
# TODO: Build your pipeline that will draw lane lines on the test_images\n",
# then save them to the test_images directory.\n",

def process_image(img):
    # Convert to gray scale
    gray = grayscale(img)
    #plt.imshow(gray, cmap='gray')
    #raise SystemExit(0)

    # Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(gray, kernel_size)

    # Canny edge detection
    low_threhold = 180
    high_threhold = 200
    edges = canny(blur_gray, low_threhold, high_threhold)

    # Region mask
    # `vertices` are hard-coded.
    imshape = edges.shape
    vertices = np.array([[(50, imshape[0]), (450, 330), (520, 330), (900, imshape[0])]])
    masked_edges = region_of_interest(edges, vertices)

    # Color
    #cv2.inRange()

    # Hough transform lines image
    rho = 1
    theta = np.pi/180
    threhold = 10
    min_line_length = 2
    max_line_gap = 2
    line_img, lines = hough_lines(masked_edges, rho, theta, threhold,
                           min_line_length, max_line_gap)
    
    # Lane lines image
    lane_img = lane_lines(img, lines, x_left=450, x_right=520)

    # Create a \"colorful\" edge image
    color_edges_img = np.dstack((edges, edges, edges))
    
    # Final result
    result = weighted_img(lane_img, img, 0.8, 1, 0)
    return image, gray, blur_gray, edges, masked_edges, line_img, lane_img, result

In [7]:

image = mpimg.imread('../test_images/solidWhiteCurve.jpg')
results = process_image(image)
titles = ['Original', 'Gray', 'Gaussian smooth', 'Canny edge detection', 'Mask region', 'Hough transform', 'Detected lanes', 'Original + Dectected lanes']

for i in range(8):
    ax = plt.subplot(4, 2, i+1)
    plt.imshow(results[i], cmap='gray')
    plt.title(titles[i])
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()    
#plt.show()
plt.savefig('example1.jpg', dpi=600)
#mpimg.imsave('example.jpg', result)